# Concepts


Recapping, the every-visit Monte Carlo method suitable for nonstationary environments is


$ V(S_t) = V(S_t) + \alpha\left[G_t - V(S_t) \right] $

where $G_t$ is the actual return following time $t$, which is different to the immediate reward from state transition, $\alpha $ is a constant step-size paramater that defines the rate in which new information is incorporated. Let's call this method **constant-$\alpha$ MC**.

As a consequence, MC methods must wait until the end of the episode to determine the increment to $V(S_t)$. In contrast, TD methods only need to wait until the next time step as they _bootstap_ (i.e. they estimate the value of the next state/action-start pair).

$ V(S_t) = V(S_t) + \alpha\left[R_t + \gamma V(S_{t+1}) - V(S_t) \right] $

This method is called TD(0) or **one-step TD**.

#### Sample Updates & Expected Updates
We refer to TD and MC updates as **Sample Updates** becauase they invovled looking ahead to a sample successor state using the value of the sucessor and the reward along the way to compute a backed-up value. Sample Updates differ from the **Expected Updates** of DP methods in that they are based on a single sample sucessor rather than on a complete distribution of all possible sucessors

#### TD Error
The quantity in the brackets in the TD(0) update is a **sort** of error, measuring the difference between the estimated value of $S_t$ and the better estimate $R_{t+1} + \gamma V(S_{t+1})$

$\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_{t})$

### Exercise 6.1 - Not Done
If $V_t$ changes during the episode, then 

$G_t - V_t(S_t) = R_{t+1} + \gamma G_{t+1} - V_t(S_t)$

$G_t - V_t(S_t) = R_{t+1} + \gamma G_{t+1} - V_t(S_t) + \gamma V_{t+1}(S_{t+1}) - \gamma V_{t+1}(S_{t+1})$

$G_t - V_t(S_t) = R_{t+1} + \gamma V_{t+1}(S_{t+1}) - V_t(S_t) + \gamma (G_{t+1} - V_{t+1}(S_{t+1}))$


### Exercise 6.2
On both TD and MC methods, we are dealing with Marcov Chain type of problems, so the value of a states is entirely dependent on the immediate return of the state and the value of the following state. If the policy is such that only a small subset of the states need to be re-estimated and the actions that the policy take don't change much outside those states, then the bootstraping $R + \gamma V(S_{t+1})$ will provide a precise enough and a much faster estimation of the value of those states than MC would

#### Advantages x Disadvantages of TD to MC and DP
- TD methods dont require a model of the environment
- With Monte Carlo methods one must wait until the end of the episode, because only then is the return known, whereas with TD methods one need to wait only one time step. Suprisingly often this turns out to be a critical consideration, because some epidodes can be extensively long.


#### But do TD Methods converge?
Yes, they have been proven to converge to $v_\pi$ in the mean for a constant step-size parameter if ut us the usual stocahstic approximation conditions

## Example 6.2 - Random Walk

In [1]:
import numpy as np
import pandas as pd
import random
import plotly.express as px

In [110]:
states = ["A", "B", "C", "D", "E", "Terminal"]
actions = ["left", "right"]
exampleTrueStateValues = {
    "A": 1/6,
    "B": 2/6,
    "C": 3/6,
    "D": 4/6,
    "E": 5/6,
    "Terminal": 0
}

def worldModelExample(state, action):
    """
    Represent the world. For each action return the reward and the next state
    """
    world = {
        "A":{
            "left": {
                "reward": 0,
                "nextState": "Terminal"
            },
            "right": {
                "reward": 0,
                "nextState": "B"
            }
        },
        "B":{
            "left": {
                "reward": 0,
                "nextState": "A"
            },
            "right": {
                "reward": 0,
                "nextState": "C"
            }
        },
        "C":{
            "left": {
                "reward": 0,
                "nextState": "B"
            },
            "right": {
                "reward": 0,
                "nextState": "D"
            }
        },
        "D":{
            "left": {
                "reward": 0,
                "nextState": "C"
            },
            "right": {
                "reward": 0,
                "nextState": "E"
            }
        },
        "E":{
            "left": {
                "reward": 0,
                "nextState": "D"
            },
            "right": {
                "reward": 1,
                "nextState": "Terminal"
            }
        },
        "Terminal":{
            "left": {
                "reward": 0,
                "nextState": "Terminal"
            },
            "right": {
                "reward": 0,
                "nextState": "Terminal"
            }
        }
    }
    return world[state][action]["reward"], world[state][action]["nextState"]

def policy(state, actions):
    """
    Method that represents the policy. In this case of Markow reward process, we just randomly pick an action
    """
    return(random.choice(actions))


def calculateRMS(estimatedStateValues: dict, trueStateValues: dict):
    error = 0
    for key in estimatedStateValues.keys():
        error = error + (estimatedStateValues[key] - trueStateValues[key])**2
    return np.sqrt(error)


In [132]:
def TD0(states: list,
        actions: list,
        policy, 
        worldModel, 
        alpha: float,
        gamma: float,
        initialStatesValues: float = 0,
        episodes: int = 100,
        trueStateValues: dict = None):
    """Function to implement a TD(0) algorithm"""
    # Initialize V(S)
    stateValues = dict.fromkeys(states, initialStatesValues)
    stateValues["Terminal"] = 0 # have to ensure that the terminal state always has estimated value 0, as nothing happens afterwards
    
    if trueStateValues:
        errorEstimates = []
    
    for ep in range(episodes):
        S = "C" # on this example, the initial state is always "C"
        while S != "Terminal":
            A = policy(S, actions)
            reward, nextS = worldModelExample(S, A)
            stateValues[S] = stateValues[S] + alpha*(reward + gamma*stateValues[nextS] - stateValues[S])
            S = nextS
            
        if trueStateValues:
            errorEstimates.append(calculateRMS(stateValues, trueStateValues))

    return stateValues, errorEstimates

In [133]:
estimatedStatesValues, errors  = TD0(states, 
                                     actions,
                                     policy,
                                     worldModelExample,
                                     0.1,
                                     1,
                                     initialStatesValues=0.5,
                                     episodes=100,
                                     trueStateValues=exampleTrueStateValues
                                    )

In [185]:
numberOfEpisodes = 300
alphasToTest = [0.01, 0.02, 0.03, 0.05, 0.1, 0.15, 0.2]
plotDF = [] # pandas DF that will whole the values for the RMS of each alpha

for alpha in alphasToTest:
    estimatedStatesValues, errors  = TD0(states, 
                                     actions,
                                     policy,
                                     worldModelExample,
                                     alpha,
                                     1,
                                     initialStatesValues=0.5,
                                     episodes=numberOfEpisodes,
                                     trueStateValues=exampleTrueStateValues
                                    )
    errorDF = pd.DataFrame(
        list(zip(range(numberOfEpisodes), errors)), 
        columns = ["episode", "error"]
    )
    errorDF["alpha"] = alpha
    plotDF.append(errorDF)


plotDF = pd.concat(plotDF)

In [186]:
fig = px.line(
    plotDF,
    color="alpha",
    y="error",
    x="episode",
    height=400,
    width=800,
    template="plotly_white",
    labels={"model": "Model"},
    color_discrete_sequence=px.colors.qualitative.Set2,
    title="Empirical RMS Error, average over states"
)
fig.add_hline(y=0.0, line=dict(color="rgba(0,0,0,0.25)", width=1))
fig.update_yaxes(range=[0, 0.5])

fig.show()

### Exercise 6.3 
As in this situation all states are intialized with the same value of 0.5 and all state transitions that don't lead to the terminal state have reward 0, the **TD Error** for all states besides "A" and "E" will **always be 0** for the first episode. Given that in the example the only episode that had the estimated value updated was "A", we can be sure that the last state on that episode was "A".


### Exercise 6.4 - Would the decision of which algorithm, MC or TD, change depending on which alpha we tested?
In this specific problem, it likely wouldn't, as it is clear that for $\alpha = 0.05$, we have a rate of RMS Error reduction that is both more stable and smaller at the end of the 100 episodes *AND* converges faster than even the fastest MC

### Exercise 6.5 - Do you think the 'noisy' behaviour of the TD for high $\alpha$ is just because of the initial value?
The "noisiness" of the RMS Error for high alpha happens because it is taking each new observation of the value of a state based on the next observed state it had, independently of how many observations it already had. With every new step, $V(S)$ will be updated in a much greater intensity than it should, given the number of observations it already had. The initial values for the states only influences after how many episodes it takes for this "noisiness" to appear.

This is quite a common problem in non-linear control and a situation in which is normally caused when the updates don't decrease in the appropriate intensity as the one value comes closer to the stabiity point.


### Exercise 6.6 - Describe 2 ways in which we could know the true values of the states
There are at least 3 ways that I can immeditely think of to find the true values of the states
- Calculating it analytically
- use Dynamic Programming, as we have the model of the world and it will be more robust than the TD
- use Monte Carlo method with a very large number of episodes, as MC will provide (in infinity) the actual expected value of each state

For this problem, I believe the best solution is using MC. Just easier

#### Batch Updating
Is when we have finite amount of experience and we present the experience repeatedly until the method converges upon an answer. This is called **batch updating** because updates are made only after processing each complete *batch* of training data.

TD(0) converges deterministically to a single answer independent of the step-size parameter, $\alpha$, as long as $\alpha$ is chosen to be sufficiently small. The constant-$\alpha$ MC method also converges deterministically under the same condition, **but to a different answer**


#### Questions
    1) How much is "sufficiently small"? How can we know for a given problem is it is small enough?
    2) How different are the TD(0) and constant-$\alpha$ MC when doing batch sampling? Will this result in a different estimated optimal policy?
    3) Theoretically we would like to arrive at the optimal policy given the data presented, so shouldn't we always do batch sampling?

Batch Monte Carlo methods always find the esimate that minimize the mean-squared error on the training set, whereas batch TD(0) always find the estimate that would be exactly correct for the maximum-likehood model of the Markov process. 

In general **maximum-likelihood estimate** of a parameter is the parameter value where the probability of generating the data is the greatest.

Given this model, we can compute the estimate of the value function thay would be exactly correct if the model were exactly correct. This is called **certainty-equivalence estimate**

## Sarsa: On-policy TD Control

The first step is to learn an action-value function rather than a state-valeue function:

$ G(S_t, A_t) = G(S_t, A_t) + \alpha\left[R_{t+1} + \gamma G(S_{t+1}, A_{t+1}) - G(S_t, A_t) \right] $


The update is done after every transition from a nonterminal state $S_t$. This method is called Sarsa because of the quintuple $(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$. The convergence properties of the Sarsa algorithm depend on the nature of the police dependence on $Q$. For $\epsilon$-greey or $\epsilon$-soft policies, Sarsa converges with probability 1 to an optimal policy and action-value as long as all state-actions are visitued an infinite number of times.

### Exercise 6.8 - Not Done

Given the action-value form of the TD Error: 
$\delta_t = R_{t+1} + \gamma G(S_{t+1}, A_{t+1}) - G(S_t, A_t)$



## Sarsa algorithm and Example 6.5 : Windy Gridworld - Not Done

In [4]:
def Sarsa(
    states: list,
    actions: list,
    policy, 
    worldModel: dict, 
    alpha: float,
    gamma: float,
    initialState,
    initialStatesActionValues: float = 0,
    episodes: int = 100):
    """Function to implement a Sarsa"""
    
    actionsValues = dict.fromkeys(actions, initialStatesActionValues)
    stateActionsValues = dict.fromkeys(states, actionsValues)
    
    # have to ensure that the terminal state always has estimated value 0, as nothing happens afterwards
    for action in actions:
        stateActionsValues["Terminal"][action] = 0
    
    # Holds how long each time episode took to end
    timeStepsList = [] 
    
    for ep in range(episodes):
        time = 0
        S = initialState 
        A = policy(S, actions)
        while S != "Terminal":
            
            # Do action
            reward, nextS = worldModelExample(S, A)
            
            # Choose next action
            nextA = policy(nextS, Q)
            
            # Update Q(S_t, A_t)
            stateActionsValues[S][A] = (
                stateActionsValues[S][A] + alpha*(reward + gamma*stateActionsValues[nextS][nextA] - stateActionsValues[S][A])
            )
            S = nextS
            A = nextA
            time += 1
            
        # Save how long the episode was
        timeStepsList.append(time)
            
    return stateActionsValues, time

## Q-Learning and Expected Sarsa

We can easily create some variations of Sarsa by changing the related next (state, action) value we use:

- Sarsa: $ G(S_t, A_t) = G(S_t, A_t) + \alpha\left[R_{t+1} + \gamma G(S_{t+1}, A_{t+1}) - G(S_t, A_t) \right] $
- Q-Learning: $ G(S_t, A_t) = G(S_t, A_t) + \alpha\left[R_{t+1} + \gamma max_A\left(G(S_{t+1}, A_{t+1})\right) - G(S_t, A_t) \right] $
- Expected Sarsa: $ G(S_t, A_t) = G(S_t, A_t) + \alpha\left[R_{t+1} + \gamma \mathbb{E}[G(S_{t+1}, A_{t+1})] - G(S_t, A_t) \right] $


### Maximization Bias & Double Learning
In Sarsa (as it normally uses $\epsilon$-greedy policies) and Q-Learning, a maxium over estimated values is used implicitily as an estimate of the maximum value, **but this can lead to a significant positive bias**.

To avoid this, we can use the idea of **double learning**, in which for each (state, action) we have 2 estimates for $Q(S,A)$ called $Q_1$ and $Q_2$. We use one estimate, say $Q_1$ to deterinze the maximizing action $A^* = argmax_a(Q_1(a))$, and the other, $Q_2$, to provide the estimate of its value $Q_2(A^*) = Q_2(argmax_a(Q_1(a)))$. This estimate will be unbiased in the sense that $\mathbb{E}(Q_2(A^*))=q(A^*)$


### Afterstates
In some games, it is more useful to use the *afterstate* (i.e. the state after the action is taken) because it is deterministic and helps in improving the learning speed of Q and V. Think of the tic-tact-toe game

'A'